In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/gws/merged_gw.csv")

In [3]:
def accumulated(df):
    # Add a column 'n' with all values set to 1
    df['n'] = 1
    
    # Group by 'name' and 'team'
    grouped = df.groupby(['name', 'team'])
    
    # Define the columns to apply the cumulative sum and then subtract the original value
    cols_to_accumulate = ['goals_scored', 'assists', 'ict_index', 'goals_conceded', 'minutes', 'own_goals', 'bps', 'clean_sheets', 'bonus']
    
    # Apply the cumulative sum and subtraction
    for col in cols_to_accumulate:
        df[col + '_cum'] = grouped[col].cumsum() - df[col]
    
    # For 'total_points', just the cumulative sum is needed
    df['total_points_cum'] = grouped['total_points'].cumsum()
    
    # Select the desired columns
    df = df[['name', 'team', 'position', 'value', 'goals_scored_cum', 'assists_cum', 'ict_index_cum', 'goals_conceded_cum', 'minutes_cum', 'own_goals_cum', 'total_points_cum', 'bps_cum', 'bonus_cum', 'clean_sheets_cum', 'GW', 'kickoff_time']]
    
    # Filter rows where 'ict_index_cum' > 0
    #df = df[df['ict_index_cum'] > 0]
    
    # Drop the temporary 'n' column
    #df.drop(columns=['n'], inplace=True)
    
    # Rename the accumulated columns back to their original names for clarity
    df.rename(columns={col + '_cum': col for col in cols_to_accumulate}, inplace=True)
    df.rename(columns={'total_points_cum': 'total_points'}, inplace=True)
    
    return df

In [4]:
cumulative_df = accumulated(data)

In [5]:
cumulative_df

,name,team,position,value,goals_scored,assists,ict_index,goals_conceded,minutes,own_goals,total_points,bps,bonus,clean_sheets,GW,kickoff_time
0,Femi Seriki,Sheffield Utd,DEF,40,0,0,0.0,0,0,0,0,0,0,0,1,2023-08-12T14:00:00Z
1,Jack Hinshelwood,Brighton,MID,45,0,0,0.0,0,0,0,0,0,0,0,1,2023-08-12T14:00:00Z
2,Jadon Sancho,Man Utd,MID,70,0,0,0.0,0,0,0,1,0,0,0,1,2023-08-14T19:00:00Z
3,Rhys Norrington-Davies,Sheffield Utd,DEF,40,0,0,0.0,0,0,0,0,0,0,0,1,2023-08-12T14:00:00Z
4,Vitaly Janelt,Brentford,MID,55,0,0,0.0,0,0,0,2,0,0,0,1,2023-08-13T13:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21422,Hákon Valdimarsson,Brentford,GK,40,0,0,0.0,0,0,0,0,0,0,0,29,2024-03-16T15:00:00Z
21423,Andros Townsend,Luton,MID,50,1,4,50.1,20,835,0,51,167,3,3,29,2024-03-16T15:00:00Z
21424,Jacob Brown,Luton,FWD,49,2,1,49.1,26,1013,0,39,115,3,1,29,2024-03-16T15:00:00Z
21425,Braian Ojeda Rodríguez,Nott'm Forest,MID,45,0,0,0.0,0,0,0,0,0,0,0,29,2024-03-16T15:00:00Z


In [6]:
cumulative_df["date"] = pd.to_datetime(cumulative_df["kickoff_time"]).dt.date
cumulative_df.index = cumulative_df["date"]
cumulative_df = cumulative_df.sort_index()
date_range = pd.date_range(start=cumulative_df.index.min(), end = cumulative_df.index.max())
cumulative_df["name_team"] = cumulative_df["name"] + "_" + cumulative_df["team"]
new_df = pd.DataFrame()
for combo in list(cumulative_df["name_team"].unique()):
    new_data = cumulative_df[cumulative_df["name_team"] == combo]
    post = new_data["position"].unique()[0]
    name, team = combo.split("_")[0], combo.split("_")[1]
    new_data_reindexed = new_data.reindex(date_range)
    new_data_reindexed["name"] = name
    new_data_reindexed["team"] = team
    new_data_reindexed["position"] = post
    new_data_reindexed["date"] = new_data_reindexed.index
    new_data_reindexed = new_data_reindexed.ffill(axis=0)
    new_data_reindexed = new_data_reindexed.dropna()
    new_data_reindexed = new_data_reindexed.reset_index(drop=True)
    
    new_df = pd.concat([new_df, new_data_reindexed], axis = 0).reset_index(drop=True)

In [7]:
latest_df = new_df[new_df["date"]==max(new_df["date"])][["name", "team", "total_points", "date", "position", "value", "GW", "minutes"]].reset_index(drop=True)

In [8]:
len(latest_df)

866

In [9]:
df2 = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/players_raw.csv")
df2["name"] = df2["first_name"] + " " + df2["second_name"]
df2 = df2[["name", "news_added", "news", "points_per_game", "team", "form",'chance_of_playing_next_round',
       'chance_of_playing_this_round']]
df3 = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/teams.csv")
df_new = df2.merge(df3, left_on = "team", right_on="id")
df_new = df_new.rename(columns={"name_x":"name", "team":"team_code","name_y":"team", "form_x":"form"})
df_new=df_new[["name", "team", "news", "news_added", "points_per_game", "form", 'chance_of_playing_next_round',
       'chance_of_playing_this_round']]
tot_df = df_new.merge(latest_df, on=["name","team"])
tot_df["id"] = tot_df.index + 1

In [10]:
df_value = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/cleaned_players.csv")
df_value["name"] = df_value["first_name"] + " " + df_value["second_name"]
df_value["id"] = df_value.index + 1
df_value = df_value[["id", "name", "now_cost"]]

In [11]:
tot_df = tot_df.merge(df_value, on = "id")

In [12]:
tot_df

,name_x,team,news,news_added,points_per_game,form,chance_of_playing_next_round,chance_of_playing_this_round,total_points,date,position,value,GW,minutes,id,name_y,now_cost
0,Folarin Balogun,Arsenal,Transferred to Monaco,2023-08-31T08:55:15.272751Z,0.0,0.0,0,0,0.0,2024-03-17,FWD,44.0,28.0,0.0,1,Folarin Balogun,44
1,Cédric Alves Soares,Arsenal,NaN,None,1.0,0.3,None,None,3.0,2024-03-17,DEF,38.0,28.0,59.0,2,Cédric Alves Soares,38
2,Mohamed Elneny,Arsenal,NaN,2023-12-30T22:30:09.113694Z,2.0,0.3,100,100,6.0,2024-03-17,MID,44.0,28.0,25.0,3,Mohamed Elneny,44
3,Fábio Ferreira Vieira,Arsenal,NaN,2023-12-07T12:00:08.489004Z,2.4,0.3,100,100,22.0,2024-03-17,MID,54.0,28.0,281.0,4,Fábio Ferreira Vieira,54
4,Gabriel dos Santos Magalhães,Arsenal,Knock - 75% chance of playing,2024-03-18T22:00:08.938811Z,4.0,3.0,75,100,103.0,2024-03-17,DEF,53.0,28.0,2084.0,5,Gabriel dos Santos Magalhães,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,Harvey Griffiths,Wolves,NaN,None,0.0,0.0,None,None,0.0,2024-03-17,MID,45.0,28.0,0.0,834,Harvey Griffiths,45
834,Yerson Mosquera,Wolves,On loan to Villarreal until the end of the season,2024-01-25T09:07:40.695577Z,0.0,0.0,0,0,0.0,2024-03-17,DEF,40.0,28.0,0.0,835,Yerson Mosquera,40
835,Noha Lemina,Wolves,NaN,None,0.0,0.0,None,None,0.0,2024-03-17,MID,50.0,28.0,0.0,836,Noha Lemina,50
836,Wesley Okoduwa,Wolves,NaN,None,0.0,0.0,None,None,0.0,2024-03-17,DEF,40.0,28.0,0.0,837,Wesley Okoduwa,40


In [14]:
tot_df = tot_df[tot_df["news"].isna()].reset_index(drop=True)
#tot_df = tot_df[~tot_df["chance_of_playing_this_round"].isin(["0", "None"])].reset_index(drop=True)

In [15]:
tot_df

,name_x,team,news,news_added,points_per_game,form,chance_of_playing_next_round,chance_of_playing_this_round,total_points,date,position,value,GW,minutes,id,name_y,now_cost
0,Cédric Alves Soares,Arsenal,NaN,None,1.0,0.3,None,None,3.0,2024-03-17,DEF,38.0,28.0,59.0,2,Cédric Alves Soares,38
1,Mohamed Elneny,Arsenal,NaN,2023-12-30T22:30:09.113694Z,2.0,0.3,100,100,6.0,2024-03-17,MID,44.0,28.0,25.0,3,Mohamed Elneny,44
2,Fábio Ferreira Vieira,Arsenal,NaN,2023-12-07T12:00:08.489004Z,2.4,0.3,100,100,22.0,2024-03-17,MID,54.0,28.0,281.0,4,Fábio Ferreira Vieira,54
3,Kai Havertz,Arsenal,NaN,2023-12-23T20:00:08.458889Z,4.1,11.7,100,100,110.0,2024-03-17,MID,71.0,28.0,1683.0,6,Kai Havertz,72
4,Gabriel Fernando de Jesus,Arsenal,NaN,2024-02-05T08:30:09.036230Z,3.4,0.7,100,100,65.0,2024-03-17,FWD,78.0,28.0,1117.0,7,Gabriel Fernando de Jesus,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,Ty Barnett,Wolves,NaN,None,0.0,0.0,None,None,0.0,2024-03-17,MID,45.0,28.0,0.0,833,Ty Barnett,45
505,Harvey Griffiths,Wolves,NaN,None,0.0,0.0,None,None,0.0,2024-03-17,MID,45.0,28.0,0.0,834,Harvey Griffiths,45
506,Noha Lemina,Wolves,NaN,None,0.0,0.0,None,None,0.0,2024-03-17,MID,50.0,28.0,0.0,836,Noha Lemina,50
507,Wesley Okoduwa,Wolves,NaN,None,0.0,0.0,None,None,0.0,2024-03-17,DEF,40.0,28.0,0.0,837,Wesley Okoduwa,40


In [16]:
tot_df = tot_df[(tot_df["points_per_game"] > 0)].reset_index(drop=True)
tot_df = tot_df.drop(["value", "name_y"], axis = 1)
tot_df = tot_df.rename(columns={"now_cost":"value", "name_x":"name"})
tot_df = tot_df[["name", "team", "total_points", "date", "position", "value", "GW", "minutes", "points_per_game"]]

In [17]:
tot_df = tot_df[tot_df["minutes"]>=450].reset_index(drop=True)

In [18]:
tot_df.to_excel("total_data.xlsx")

In [19]:
gk = tot_df[tot_df["position"]=="GK"].sort_values("points_per_game", ascending=False).reset_index(drop=True).head(10)
defenders = tot_df[tot_df["position"]=="DEF"].sort_values("points_per_game", ascending=False).reset_index(drop=True).head(30)
midfielders = tot_df[tot_df["position"]=="MID"].sort_values("points_per_game", ascending=False).reset_index(drop=True).head(30)
forwards = tot_df[tot_df["position"]=="FWD"].sort_values("points_per_game", ascending=False).reset_index(drop=True).head(30)

total_df = pd.concat([gk, defenders, midfielders, forwards], axis = 0).sort_values("position", ascending=False).reset_index(drop=True)

In [20]:
total_df.to_excel("use_data.xlsx")

In [21]:
tot_df

,name,team,total_points,date,position,value,GW,minutes,points_per_game
0,Kai Havertz,Arsenal,110.0,2024-03-17,MID,72,28.0,1683.0,4.1
1,Gabriel Fernando de Jesus,Arsenal,65.0,2024-03-17,FWD,78,28.0,1117.0,3.4
2,Jorge Luiz Frello Filho,Arsenal,30.0,2024-03-17,MID,53,28.0,654.0,1.7
3,Jakub Kiwior,Arsenal,53.0,2024-03-17,DEF,44,28.0,707.0,3.3
4,Eddie Nketiah,Arsenal,68.0,2024-03-17,FWD,51,28.0,1037.0,2.8
...,...,...,...,...,...,...,...,...,...
281,José Malheiro de Sá,Wolves,77.0,2024-03-17,GK,50,28.0,2137.0,3.0
282,Pablo Sarabia,Wolves,79.0,2024-03-17,MID,47,28.0,1178.0,3.8
283,Nélson Cabral Semedo,Wolves,59.0,2024-03-17,DEF,45,28.0,2257.0,2.2
284,Toti António Gomes,Wolves,62.0,2024-03-17,DEF,44,28.0,1827.0,2.5


In [22]:
total_df[total_df["team"]=="Liverpool"]

,name,team,total_points,date,position,value,GW,minutes,points_per_game
1,Mohamed Salah,Liverpool,168.0,2024-03-17,MID,131,28.0,1786.0,7.6
24,Luis Díaz,Liverpool,102.0,2024-03-17,MID,75,28.0,1742.0,3.8
27,Dominik Szoboszlai,Liverpool,85.0,2024-03-17,MID,71,28.0,1673.0,3.7
33,Caoimhin Kelleher,Liverpool,27.0,2024-03-17,GK,38,28.0,540.0,3.9
55,Cody Gakpo,Liverpool,69.0,2024-03-17,FWD,72,28.0,1136.0,2.7
93,Virgil van Dijk,Liverpool,100.0,2024-03-17,DEF,65,28.0,2187.0,3.8
96,Konstantinos Tsimikas,Liverpool,47.0,2024-03-17,DEF,45,28.0,669.0,3.9
